In [2]:
import argparse
import gym
import numpy as np
from itertools import count
from collections import namedtuple

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

args = {
    'gamma': 0.99,
    'seed': 543,
    'render': True,
    'log_interval': 10
}

env = gym.make('CartPole-v0')
env.seed(args['seed'])
torch.manual_seed(args['seed'])
print("action_space={}".format(env.action_space))

SavedAction = namedtuple('SavedAction', ['log_prob', 'value'])


class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(4, 128)
        self.action_head = nn.Linear(128, 2)
        self.value_head = nn.Linear(128, 1)

        self.saved_actions = []
        self.rewards = []

    def forward(self, x):
        x = F.relu(self.affine1(x))
        action_scores = self.action_head(x)
        state_values = self.value_head(x)
        return F.softmax(action_scores, dim=-1), state_values


model = Policy()
optimizer = optim.Adam(model.parameters(), lr=3e-2)
eps = np.finfo(np.float32).eps.item()


def select_action(state):
    state = torch.from_numpy(state).float()
    probs, state_value = model(state)
    m = Categorical(probs)
    action = m.sample()
    model.saved_actions.append(SavedAction(m.log_prob(action), state_value))
    return action.item()


def finish_episode():
    R = 0
    saved_actions = model.saved_actions
    policy_losses = []
    value_losses = []
    rewards = []
    for r in model.rewards[::-1]:
        R = r + args['gamma'] * R
        rewards.insert(0, R)
    rewards = torch.tensor(rewards)
    rewards = (rewards - rewards.mean()) / (rewards.std() + eps)
    for (log_prob, value), r in zip(saved_actions, rewards):
        reward = r - value.item()
        policy_losses.append(-log_prob * reward)
        value_losses.append(F.smooth_l1_loss(value, torch.tensor([r])))
    optimizer.zero_grad()
    loss = torch.stack(policy_losses).sum() + torch.stack(value_losses).sum()
    loss.backward()
    optimizer.step()
    del model.rewards[:]
    del model.saved_actions[:]


def main():
    running_reward = 10
    for i_episode in count(1):
        state = env.reset()
        for t in range(10000):  # Don't infinite loop while learning
            action = select_action(state)
            state, reward, done, _ = env.step(action)
            if args['render']:
                env.render()
            model.rewards.append(reward)
            if done:
                break

        running_reward = running_reward * 0.99 + t * 0.01
        finish_episode()
        if i_episode % args['log_interval'] == 0:
            print('Episode {}\tLast length: {:5d}\tAverage length: {:.2f}'.format(
                i_episode, t, running_reward))
        if running_reward > env.spec.reward_threshold:
            print("Solved! Running reward is now {} and "
                  "the last episode runs to {} time steps!".format(running_reward, t))
            break


print("threshold={} \n".format(env.spec.reward_threshold))

main()


action_space=Discrete(2)
threshold=195.0 

Episode 10	Last length:     8	Average length: 10.07
Episode 20	Last length:     9	Average length: 10.90
Episode 30	Last length:    10	Average length: 10.70
Episode 40	Last length:     9	Average length: 10.49
Episode 50	Last length:    11	Average length: 10.34
Episode 60	Last length:     8	Average length: 10.20
Episode 70	Last length:     8	Average length: 10.14
Episode 80	Last length:    11	Average length: 10.19
Episode 90	Last length:     9	Average length: 10.12
Episode 100	Last length:    31	Average length: 10.42
Episode 110	Last length:    11	Average length: 10.91
Episode 120	Last length:    25	Average length: 12.45
Episode 130	Last length:    37	Average length: 15.14
Episode 140	Last length:    37	Average length: 19.75
Episode 150	Last length:    23	Average length: 19.90
Episode 160	Last length:    10	Average length: 19.80
Episode 170	Last length:    36	Average length: 20.36
Episode 180	Last length:    66	Average length: 25.60
Episode 190	

Episode 1520	Last length:   191	Average length: 136.33
Episode 1530	Last length:   199	Average length: 141.61
Episode 1540	Last length:   199	Average length: 146.77
Episode 1550	Last length:   199	Average length: 151.53
Episode 1560	Last length:   199	Average length: 154.25
Episode 1570	Last length:   167	Average length: 156.71
Episode 1580	Last length:   149	Average length: 157.10
Episode 1590	Last length:   145	Average length: 154.04
Episode 1600	Last length:   151	Average length: 154.13
Episode 1610	Last length:   146	Average length: 155.13
Episode 1620	Last length:   161	Average length: 154.97
Episode 1630	Last length:   174	Average length: 157.46
Episode 1640	Last length:   199	Average length: 161.08
Episode 1650	Last length:   199	Average length: 164.65
Episode 1660	Last length:   199	Average length: 167.83
Episode 1670	Last length:   199	Average length: 170.81
Episode 1680	Last length:   199	Average length: 173.51
Episode 1690	Last length:   199	Average length: 175.95
Episode 17